In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [107]:
#juliette cd
cd='/content/drive/MyDrive/Drexel Data Science Masters/DSCI521 - Data Analysis and Interpretation/DSCI521 Group Work/DSCI521 Project/'

In [105]:
#DATA PREP
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

df = pd.read_excel(cd+"enrollment_expenditure_covar_imp_df.xlsx")
df = df.copy()

df.rename(columns={'STATE': 'state',
                   'YEAR': 'year',
                   'Total   Computable': 'expenditure',
                   'Average Total Medicaid Enrollment': 'enrollment'}, inplace=True)
df.sort_values(by=['state', 'year'], inplace=True)

# standardize expenditure and enrollment across states of different populations.
df['expenditure_pc'] = df['expenditure'] / df['totalPop']
df['enrollment_pc'] = df['enrollment'] / df['totalPop']


# Log-transformation
df['log_expenditure'] = np.log(df['expenditure_pc'])
df['log_enrollment'] = np.log(df['enrollment_pc'])

# Compute year-over-year differences
df['d_log_expenditure'] = df.groupby('state')['log_expenditure'].diff()
df['d_log_enrollment'] = df.groupby('state')['log_enrollment'].diff()

# control variables: calculate year-over-year differences
df['d_unemployment_rate'] = df.groupby('state')['unemployed'].diff()
df['d_median_income'] = df.groupby('state')['medianIncome'].diff()


# COVID dummy variable:
df['COVID_dummy_2020'] = df['year'].apply(lambda x: 1 if x in [2020] else 0)
df['COVID_dummy_2021'] = df['year'].apply(lambda x: 1 if x in [2021] else 0)
df['COVID_dummy_2022'] = df['year'].apply(lambda x: 1 if x in [2022] else 0)

# interaction term between the differenced log enrollment and the COVID dummy.
df['d_log_expenditure_COVID_2020'] = df['d_log_expenditure'] * df['COVID_dummy_2020']
df['d_log_expenditure_COVID_2021'] = df['d_log_expenditure'] * df['COVID_dummy_2021']
df['d_log_expenditure_COVID_2022'] = df['d_log_expenditure'] * df['COVID_dummy_2022']

#mke int state var
state_mapping = {"Alabama": 0,
"Alaska": 1,
"Arizona": 2,
"Arkansas": 3,
"California": 4,
"Colorado": 5,
"Connecticut": 6,
"Delaware": 7,
"District of Columbia": 8,
"Florida": 9,
"Georgia": 10,
"Hawaii": 11,
"Idaho": 12,
"Illinois": 13,
"Indiana": 14,
"Iowa": 15,
"Kansas": 16,
"Kentucky": 17,
"Louisiana": 18,
"Maine": 19,
"Maryland": 20,
"Massachusetts": 21,
"Michigan": 22,
"Minnesota": 23,
"Mississippi": 24,
"Missouri": 25,
"Montana": 26,
"Nebraska": 27,
"Nevada": 28,
"New Hampshire": 29,
"New Jersey": 30,
"New Mexico": 31,
"New York": 32,
"North Carolina": 33,
"North Dakota": 34,
"Ohio": 35,
"Oklahoma": 36,
"Oregon": 37,
"Pennsylvania": 38,
"Rhode Island": 39,
"South Carolina": 40,
"South Dakota": 41,
"Tennessee": 42,
"Texas": 43,
"Utah": 44,
"Vermont": 45,
"Virginia": 46,
"Washington": 47,
"West Virginia": 48,
"Wisconsin": 49,
"Wyoming": 50}
df['state_int'] = df['state'].map(state_mapping)

#remove state
df = df.drop('state', axis=1)

# Drop rows the first observation per state
df_diff = df.dropna(subset=['d_log_expenditure', 'd_log_enrollment'])

# Create year dummy variables to capture common time shocks.
df_diff = pd.get_dummies(df_diff, columns=['year'], drop_first=True)

In [108]:
#MODEL
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error

# Model specification in first-difference form:
formula = 'd_log_enrollment ~ d_log_expenditure + COVID_dummy_2020 + COVID_dummy_2021 + COVID_dummy_2022 + d_log_expenditure_COVID_2020 + d_log_expenditure_COVID_2021 + d_log_expenditure_COVID_2022'

# Control variables
control_vars = []
control_vars.append('d_unemployment_rate')
control_vars.append('d_median_income')

formula += ' + ' + ' + '.join(control_vars)

print("Regression formula:")
print(formula)

#train_test_split(x, y, test_size, random_state) (assigns 33% to test data)
df_train, df_test = train_test_split(df_diff, test_size=0.33, random_state=42)

# Estimate the OLS model
model = smf.ols(formula=formula, data=df_train).fit(cov_type='cluster', cov_kwds={'groups': df_train['state_int']})
print(model.summary())

#create x and y test datasets
yreg_test=df_test['d_log_enrollment']
xreg_test=df_test[['d_log_expenditure','COVID_dummy_2020','COVID_dummy_2021','COVID_dummy_2022','d_log_expenditure_COVID_2020','d_log_expenditure_COVID_2021','d_log_expenditure_COVID_2022','d_unemployment_rate','d_median_income']]

#make predictions
predictions = model.predict(xreg_test)

#evaluate prediction accuracy
rmse = np.sqrt(mean_squared_error(yreg_test, predictions))
print(f"Root Mean Squared Error: {rmse}")


Regression formula:
d_log_enrollment ~ d_log_expenditure + COVID_dummy_2020 + COVID_dummy_2021 + COVID_dummy_2022 + d_log_expenditure_COVID_2020 + d_log_expenditure_COVID_2021 + d_log_expenditure_COVID_2022 + d_unemployment_rate + d_median_income
                            OLS Regression Results                            
Dep. Variable:       d_log_enrollment   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     61.09
Date:                Mon, 10 Mar 2025   Prob (F-statistic):           3.83e-23
Time:                        13:31:40   Log-Likelihood:                 297.28
No. Observations:                 170   AIC:                            -574.6
Df Residuals:                     160   BIC:                            -543.2
Df Model:                           9                                         
Covariance Type:              cluster     

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 8
  warnings.warn('covariance of constraints does not have full '
